# 03 - Neo4j Knowledge Graphs

Work with graph databases using Neo4j.

In [ ]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
import pandas as pd

load_dotenv()
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))
)

## 1. Create Sample Graph

In [ ]:
with driver.session() as session:
    session.run("""
        MERGE (stack:System {name: 'GenAI Vanilla Stack'})
        MERGE (ollama:Service {name: 'Ollama', type: 'LLM'})
        MERGE (weaviate:Service {name: 'Weaviate', type: 'VectorDB'})
        MERGE (neo4j:Service {name: 'Neo4j', type: 'GraphDB'})
        MERGE (jupyter:Service {name: 'JupyterHub', type: 'IDE'})
        
        MERGE (stack)-[:INCLUDES]->(ollama)
        MERGE (stack)-[:INCLUDES]->(weaviate)
        MERGE (stack)-[:INCLUDES]->(neo4j)
        MERGE (stack)-[:INCLUDES]->(jupyter)
        
        MERGE (jupyter)-[:CONNECTS_TO]->(ollama)
        MERGE (jupyter)-[:CONNECTS_TO]->(weaviate)
        MERGE (jupyter)-[:CONNECTS_TO]->(neo4j)
    """)
print("✅ Graph created")

## 2. Query Graph

In [ ]:
with driver.session() as session:
    result = session.run("""
        MATCH (jupyter:Service {name: 'JupyterHub'})-[:CONNECTS_TO]->(service)
        RETURN service.name as name, service.type as type
    """)
    df = pd.DataFrame([record.data() for record in result])
    display(df)

## 3. Path Finding

In [ ]:
with driver.session() as session:
    result = session.run("""
        MATCH path = (stack:System)-[*]-(service:Service {type: 'LLM'})
        RETURN [node in nodes(path) | node.name] as path
    """)
    for record in result:
        print(" → ".join(record["path"]))